<a href="https://colab.research.google.com/github/irinap9270/hg_ml/blob/main/Ch4-1.%20LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# """럭키백의 확률"""

> 럭키백에 포함된 생선의 확률은 어떻게 알 수 있을까?

이웃의 클래스 비율을 확률이라고 출력하면 되지 않을까?


In [3]:
# 데이터 준비
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')
# 처음 5개 행 출력
print(fish.head())

# 생선 종류 출력
print(pd.unique(fish['Species']))

  Species  Weight  Length  Diagonal   Height   Width
0   Bream   242.0    25.4      30.0  11.5200  4.0200
1   Bream   290.0    26.3      31.2  12.4800  4.3056
2   Bream   340.0    26.5      31.1  12.3778  4.6961
3   Bream   363.0    29.0      33.5  12.7300  4.4555
4   Bream   430.0    29.0      34.0  12.4440  5.1340
['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [23]:
# Species를 제외한 열을 입력 데이터로 사용
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()
# 타겟 데이터
fish_target = fish['Species'].to_numpy()

# 훈련 세트와 테스트 세트로 나눔
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

# 훈련 세트와 테스트 세트를 표준화 전처리
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

# 최근접 이웃의 개수를 3으로 하여 학습한 다음 정확도 출력
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


In [24]:
# 타깃값 정렬 확인(알파벳 순서)
print(kn.classes_)

# 5개 샘플에 대한 예측값 출력
print(kn.predict(test_scaled[:5]))
# 클래스별 확률값 확인
import numpy as np
proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))

# 네 번째 샘플의 최근접 이웃 확인
distances, indexes = kn.kneighbors(test_scaled[3:4])
print(train_target[indexes])

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']
[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]
[['Roach' 'Perch' 'Perch']]


3개의 최근접 이웃만 사용하기 때문에,
확률이 0, 1/3, 2/3, 1이 전부이다.

# """로지스틱 회귀"""
> 이름은 회귀이지만 분류 모델

> 선형 회귀와 동일하게 선형 방정식을 학습

로지스틱 회귀로 이진 분류를 해보자.

In [33]:
# train_target에서 도미와 빙어인 것만 True인 배열을 반환하여 인덱스로 생성
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
# 도미, 빙어에 대한 훈련 세트, 테스트 세트 생성
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

# 처음 5개 샘플 예측
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

# 처음 5개 샘플 예측
print(lr.predict(train_bream_smelt[:5]))
# 처음 5개 샘플의 각 확률 출력
print(lr.predict_proba(train_bream_smelt[:5]))

# 학습한 계수(가중치) 확인
print(lr.coef_, lr.intercept_)

# 처음 5개 샘플의 z값 출력
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions)

# z값을 시그모이드함수에 통과시켜 확률 얻기
from scipy.special import expit
print(expit(decisions))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']
[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]
[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]
[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]
[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


"""로지스틱 회귀로 다중 분류하기"""

7개의 생선을 분류해보자.

In [41]:
# C; L2규제(계수의 제곱을 규제)를 제어하는 매개변수. 작을수록 규제가 커짐
# max_iter; 반복 횟수를 지정하는 매개변수
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

# 처음 5개 샘플의 예측값 출력
print(lr.predict(test_scaled[:5]))
# 처음 5개 샘플의 예측 확률 출력
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

# 방정식의 계수와 절편의 개수 출력
print(lr.coef_.shape, lr.intercept_.shape)

0.9327731092436975
0.925
['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
(7, 5) (7,)


특성이 5개 > coef_배열의 열은 5개

coef_배열의 행과 intercept_ 7개 > 클래스마다 z값을 하나씩 계산한다.

> 가장 높은 z값을 출력하는 클래스가 예측 클래스가 된다.

소프트맥스 함수를 사용해 확률로 바꾸어보자

In [46]:
# decision_function(); z1-z7의 값을 구하는 함수
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

# 소프트맥스 함수 사용
from scipy.special import softmax
# axis=1로 지정하여 각 행(샘플)에 대해 소프트맥스를 게산함
proba = softmax(decision, axis=1)
print(np.round(proba, decimals=3))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]
[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
